# Notes from Doug

If people can take one lesson away from reading this, it’s why do we use GPUs and how is the computational model associated with GPUs more sensible for NN needs? 
Sections 5-7 are the real important ones
What parts of NN are able to be parallelized? SIMD, sending batches


Set the stage for GPUs: 
What are they, how do they work? 
Why are they good for matrix multiplication?
How is that useful for machine learning? 
How can convolution/recurrent stuff be reshaped to be used in this way? 


Section 8 can be mined for discussion topics 
What are limits and why - not enough data, or not enough computational time?
What key advances will be necessary to use larger data sets? 
OR what will make training high-performance models available to someone on their desktop? 

# 5 Deep Neural Networks(DNN)

Current trend if toward concurrency within mini-batches
 
Even if SIMD is not explicitly utilized, your code is almost garunteed to use SIMD

GPUs accel at performing matrix and vector operations 

Three periods in the history of classification neural networks:
  1. Experimentation(~1985-2010) 
  2. Growth(2010-2015)
  3. Resource Conservation(2015-present) 

# 6 Concurrency In Networks

![Parallelism schemes: Data, Model, and Pipeline](img/par_Types.PNG)


Work-Depth model:
  Work: amount of actual computation required for a specific operator
  Depth: longest chain of of data dependencies
  Lower depth operations are more trivial to parallelize

![Work-Depth model](img/tab_4.PNG)



## 6.1 Data Parallelism
  Since Most operators are independant with respect to the sample size, a single operator can be performed concurrently on different batches of the input data.
  Parallel computation does not neet to synchronize after every operation.

![data level parallelism](img/data_par.png)

How It's Done
  Partition*N* samples into minibatches
  Distrubute batches across computational resources


Scaling
 Parallel scaling is determined by the minibatch size

Batch Normalization(BN)
  Normalizing a batch requries synchronization across the batch.
  Number of times BN is required varies between architectures.
  Snce BN requires synchronization of all samples a minibatch, if minibatches are small enough to be handled independently, no additional synchronization is necessary.

## 6.2 Model Parallelism

Model parallelism means to split portions of individual layers such that they can be computed independently.
 Critical in situations where the entire model does not fit in memory.

Unline data parallelism, sample batch is distributed to all processors.

Since an individual layer is broken up between processors, the inter-layer data depenedencies require communication between the processors.
  e.g., a network comprised of 2 fully-connected layers, parallelized between 4 processors. Each processor computes 1/4th of the nodes in each layer,and then sends the results to the other 3 processors.

The amount of data dependencies determine overall performance.
*Convolutional Layer > Fully-connected Layer > Locally Connected Layer*

In case of DNNs, partition work according to dimension (i.e. C, H or W for a 4D tensor). The dependencies which in turn generate communication determines overall performance. Fully connected layers incur all-to-all. Has been proposed to mitigate this problem by making processor responsible for 2x number of nodes, with overlap. Therefore more computation but less communicantion

Convoloutional layers require a large amount of communication. In comparison, Locally connected networks need very little communication, and are an excellent choice for model parallelism



Note: Should probably (briefly)talk about tree nets as they are analagous to ensamble classifiers 


## 6.3 Pipelining

Partitioning the DNN based on depth
Each processor comptues one or more layers of the DNN, and streams the results to the next layer on another processor.

Advantages:
  Each processor only needs parameters relevant to their own slice of the NN
  Communication between processors is fixed; message source and destination are always known

Disadvantages:
  Pipeline bottnecks
  Latency


# 7 Concurrency In Training

## 7.1 Model Consistency

## 7.2 Parameter Distribution

## 7.3 Training Distribution

# 8. Optimization

## 8.1 Parameter Search 

## 8.2 Hyperparameter Search